In [ ]:
import pandas as pd  # dataset handling
import numpy as np
import geopandas as gpd  # geodataset handling
from keplergl import KeplerGl  # geospatial visualization 

import datetime
import json

from prophet import Prophet
from prophet.serialize import model_from_json, model_to_json
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

## Importing latest Sensor and Meteomatics data

In [ ]:
# load meteomatics data for the cities of interest 
cities = ['Bremen', 'Frankfurt']
# set the day of the downloaded data
day = datetime.datetime(2022, 3, 20).date()
weather = {}

for city in cities: 
    weather[city] = pd.read_csv(f'../data/Meteomatics/auto_processed_weather_forecast_{city}_{day}.csv')
    weather[city].timestamp = pd.to_datetime(weather[city].timestamp)

In [ ]:
df = pd.read_csv(f'../data/cleaned_sensors_dwd_2022-03-28.csv', index_col=0)
df.timestamp = pd.to_datetime(df.timestamp) 

In [ ]:
# soon, this will be automated, once the meteomatics data is consistent in terms of the starting timestamps
df_train = df[df.timestamp <= '2022-03-20 8:00']

## Train Prophet on all available data

In [ ]:
# set locations
locations = list(df.location_id.unique())

In [ ]:
# initialize model
seasonality_mode='additive'
yearly_seasonality=True
weekly_seasonality=True
daily_seasonality=True

growth='logistic'
n_changepoints=25
changepoint_prior_scale = 0.6  #default 0.05

temp_flag = True
press_flag = True
windsp_flag = True
winddir_flag = True
precip_flag = True

temp_prior_scale = 0.3
press_prior_scale = 0.3
windsp_prior_scale = 0.3
winddir_prior_scale = 0.3
precip_prior_scale = 0.3

if not temp_flag:
    temp_prior_scale = None
if not press_flag:
    press_prior_scale = None
if not windsp_flag:
    windsp_prior_scale = None
if not winddir_flag:
    winddir_prior_scale = None
if not precip_flag:
    precip_prior_scale = None

In [ ]:
models = {}
limits = pd.DataFrame(index=locations, columns=['cap', 'floor'])
for location in locations:
    # reduce data to one location
    df_prophet_reg = df_train.query(f'location_id == {location}')[['timestamp','PM2p5', 'temperature', 'pressure', 'wind_speed', 'wind_direction', 'precip']] 
    df_prophet_reg = df_prophet_reg.sort_values(['timestamp'], axis=0)

    # rename columns to expected format for prophet
    df_prophet_reg.rename(columns={'timestamp': 'ds', 'PM2p5': 'y', 'temperature': 'temp', 'pressure': 'press', 'wind_speed': 'windsp', 'wind_direction': 'winddir', 'precip': 'precip'}, inplace=True)

    # prophet can not handle nans in dataframe
    df_prophet_reg.dropna(inplace=True, subset=['temp', 'press', 'windsp', 'precip', 'winddir'])

    df_average = df_train.query(f'location_id == {location}')[['timestamp','PM2p5']]
    df_average['PM2p5_average'] = (df_average.PM2p5.shift(2) + df_average.PM2p5.shift(1) + df_average.PM2p5 + df_average.PM2p5.shift(-1) + df_average.PM2p5.shift(-2)) / 5

    # add cap column for to set growth = logistic
    cap = df_average.PM2p5_average.quantile(0.99)
    floor = df_average.PM2p5_average.min()
    
    limits.loc[location, 'cap'] = cap
    limits.loc[location, 'floor'] = floor

    df_prophet_reg['cap'] =  cap 
    df_prophet_reg['floor'] = floor

    model = Prophet(seasonality_mode=seasonality_mode, yearly_seasonality=yearly_seasonality, weekly_seasonality=weekly_seasonality, daily_seasonality=daily_seasonality,
        growth=growth,n_changepoints=n_changepoints, changepoint_prior_scale=changepoint_prior_scale)
    
    # add regressors 
    model.add_regressor('temp', standardize=True, prior_scale=temp_prior_scale)
    model.add_regressor('press', standardize=True, prior_scale=press_prior_scale)
    model.add_regressor('windsp', standardize=True, prior_scale=windsp_prior_scale)
    model.add_regressor('winddir', standardize=True, prior_scale=winddir_prior_scale)
    model.add_regressor('precip', standardize=True, prior_scale=precip_prior_scale)

    # fit model 
    model.fit(df_prophet_reg)

    # save model in dictionary 
    models[location] = model

# Fill future dataframe with meteomatics data

In [ ]:
def create_regressor_column(ds, train_col, test_col, location_id):
    """Get a regressor of train or test data for corresponding timestamp

    Args:
        ds (datetime): timestamp
        train_col (string): column name of regressor in train data
        test_col (string): column name of regressor in test data

    Returns:
        float: regressor value for given timestamp
    """
    
    if ds in df_prophet_reg['ds'].values:
        return df_prophet_reg[df_prophet_reg['ds'] == ds][train_col].values[0]
    elif ds in weather['Frankfurt']['timestamp'].values:
        if location_id <= 124:
            return weather['Frankfurt'][(weather['Frankfurt']['timestamp'] == ds)][test_col].values[0]
        else: 
            return weather['Bremen'][(weather['Bremen']['timestamp'] == ds)][test_col].values[0]
    else:
        return np.nan

In [ ]:
forecasts = {}
for location in locations: 
    model = models[location]
    future = model.make_future_dataframe(periods=168, freq='H')
    
    future['temp'] = future['ds'].apply(create_regressor_column, args=('temp', 'temperature', location))
    future['press'] = future['ds'].apply(create_regressor_column, args=('press', 'pressure', location))
    future['windsp'] = future['ds'].apply(create_regressor_column, args=('windsp', 'wind_speed', location))
    future['winddir'] = future['ds'].apply(create_regressor_column, args=('winddir', 'wind_direction', location))
    future['precip'] = future['ds'].apply(create_regressor_column, args=('precip', 'precip', location))

    future['cap'] = limits.loc[location, 'cap']
    future['floor'] = limits.loc[location, 'floor']

    # drop nans
    future.dropna(inplace=True)
    # predict
    forecasts[location] = model.predict(future)
        

In [ ]:
def forecast_dict_to_df(forecast: dict) -> pd.DataFrame:
    """Convert forecast dictionary to dataframe

    Args:
        forecast (dict): forecast dictionary from all locations' prophet models

    Returns:
        pd.DataFrame: DataFrame with all forecasts
    """
    forecasts = pd.DataFrame(columns=['ds', 'yhat', 'location_id'])

    for location in list(forecast.keys()):
        forecast[location]['location_id'] = location
        forecasts = pd.concat([forecasts, forecast[location]], axis=0)
    return forecasts

## Save models

In [ ]:
# save models and forecasts for later use
for location_id in locations:
    model_json = model_to_json(models[location_id])
    with open(f'../models/{day}/{location_id}_model_{day}.json', 'w') as f:
        json.dump(model_json, f)
    forecasts[location_id].to_csv(f'../models/{day}/{location_id}_forecast_{day}.csv')

## Calculate RMSE

In [ ]:
# combine forecasts into one dataframe
df_forecasts = forecast_dict_to_df(forecasts)

In [ ]:
df_eval = df[(df.timestamp >= df_forecasts.ds.min()) & (df.timestamp <= df_forecasts.ds.max())][['location_id', 'timestamp', 'PM2p5']]
df_eval.rename(columns={'timestamp': 'ds'}, inplace=True)

In [ ]:
df_rmse = df_forecasts[['location_id','ds', 'yhat']].merge(df_eval, on=['location_id', 'ds'], how='left')

In [ ]:
def rmse_per_location(df_rmse: pd.DataFrame, forecast_horizon=168):
    """Calculate RMSE per location

    Args: 
        df_rmse (pd.DataFrame): dataframe with y_true and y_hat values, timestamp and location_id
        forecast_horizon (int): forecast horizon (in hours)
    
    Returns:
        pd.DataFrame: dataframe with RMSE train and test values per location
    """
    locs = df_rmse.location_id.unique().tolist()
    rmse = pd.DataFrame(columns=['RMSE_train', 'RMSE_test', 'PM2p5_mean'], index=locs)

    for loc in locs: 
        df_loc = df_rmse[df_rmse.location_id == loc]
        df_loc.dropna(inplace=True)
        rmse.loc[loc, 'RMSE_train'] = mean_squared_error(df_loc['PM2p5'][:-forecast_horizon], df_loc['yhat'][:-forecast_horizon], squared=False)
        rmse.loc[loc, 'RMSE_test'] = mean_squared_error(df_loc['PM2p5'][-forecast_horizon:], df_loc['yhat'][-forecast_horizon:], squared= False)
        rmse.loc[loc, 'PM2p5_mean'] = df_loc['PM2p5'][:-forecast_horizon].mean()

    rmse.reset_index(inplace=True)
    rmse.rename(columns={'index': 'location_id'}, inplace=True)
    return rmse


In [ ]:
# Calculate RMSE per location for 7 days (168 hours)
rmse_locations = rmse_per_location(df_rmse)
rmse_locations

In [ ]:
rmse_locations.sort_values(by='location_id').plot(kind='scatter', figsize=(25,15), x='PM2p5_mean', y='RMSE_test')

In [ ]:
rmse_locations.mean()

## Visualization with Kepler.gl

In [ ]:
# get GPS data and merge with forecast dataframe since kepler.gl needs GPS data
df_gps = df[['location_id', 'lat', 'lon']].groupby(['location_id']).first().reset_index()

In [ ]:
df_kepler = df_forecasts.merge(df_gps, how='left', on='location_id')[['ds', 'yhat', 'location_id', 'lat', 'lon']]

In [ ]:
df_kepler.dropna(subset=['yhat'], inplace=True)

In [ ]:
# make dummies
timestamps = pd.Series(df_kepler['ds'].unique(), name='ds')

dummies = pd.DataFrame(data={
    'location_id': -1,
    'lat': [0, 90],
    'lon': [0, 90],
    'yhat': [0, 50]
})

dummies = dummies.merge(timestamps, how='cross')

df_kepler = pd.concat([df_kepler, dummies])

In [ ]:
# Adjust plotting time horizon
df_kepler = df_kepler.sort_values('ds').query('ds > "2022-03"')

In [ ]:
# Create bins for PM2p5 for Hexbin plotting
pm2p5_bins = np.append(0,np.arange(0, 50, 5))
pm2p5_labels = pm2p5_bins
pm2p5_labels[0] = -1
pm2p5_bins = np.append(pm2p5_bins, 1000)
pm2p5_bins[0] = -20

In [ ]:
print(pm2p5_bins)

In [ ]:
df_kepler['PM2p5_bins'] = pd.cut(df_kepler['yhat'], bins=pm2p5_bins, labels=pm2p5_labels).astype(int)

In [ ]:
# Make the geo DataFrame
gdf_sensors = gpd.GeoDataFrame(
    df_kepler, 
    geometry=gpd.points_from_xy(
        x=df_kepler['lon'],
        y=df_kepler['lat']
    )
)

In [ ]:
# Creating a Datetime column (Kepler is funny about datetimes)
gdf_sensors['timestamp'] = pd.to_datetime(gdf_sensors['ds'])
gdf_sensors['timestamp'] = gdf_sensors['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Selecting only columns we need
gdf_sensors = gdf_sensors[[
    'yhat', 'lon', 'lat', 'geometry', 'timestamp',  'PM2p5_bins', 'location_id'
]]

gdf_sensors

In [ ]:
gdf_sensors[gdf_sensors.yhat < 0].location_id.unique()

In [ ]:
%run config.py
map_config = config

In [ ]:
kepler_map = KeplerGl(
    height=800,
    data={
        'Sensors': gdf_sensors,
    }, config=map_config
)

In [ ]:
kepler_map

### Export for Kepler.gl or Unfolded online use

In [ ]:
df_kepler.to_csv('../data/kepler.csv', index=False)